<a href="https://colab.research.google.com/github/tanaymukherjee/Natural-Language-Processing/blob/master/08_Extension_attributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowing Extension Attributes

## Setting Attributes - I

In [1]:
import spacy

In [2]:
# Create an NLP object
from spacy.lang.en import English
nlp = English()

In [3]:
# Import the Doc class
from spacy.tokens import Doc, Span, Token

In [4]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
# Register the Token extension attribute 'is_country' with the default value False
Token.set_extension('is_country', default=False, force=True)

# Process the text and set the is_country attribute to True for the token "Spain"
doc = nlp("I live in India.")
doc[3]._.is_country = True

# Print the token text and the is_country attribute for all tokens
print([(token.text, token._.is_country) for token in doc])

[('I', False), ('live', False), ('in', False), ('India', True), ('.', False)]


**NOTE:** Remember that if you run your code more than once, you might see an error message that the extension already exists. That's because you will re-run your code in the same session. To solve this, you can set force=True on set_extension, or reload to start a new Python session. 

In [6]:
# Define the getter function that takes a token and returns its reversed text
def get_reversed(token):
    return token.text[::-1]
  
# Register the Token property extension 'reversed' with the getter get_reversed
Token.set_extension('reversed', getter=get_reversed)

# Process the text and print the reversed attribute for each token
doc = nlp("All generalizations are false, including this one.")
for token in doc:
    print('reversed:', token._.reversed)

reversed: llA
reversed: snoitazilareneg
reversed: era
reversed: eslaf
reversed: ,
reversed: gnidulcni
reversed: siht
reversed: eno
reversed: .


## Setting Attributes - II

Let's try setting some more complex attributes using getters and method extensions.

Use Doc.set_extension to register 'has_number' (getter get_has_number) and print its value.

In [7]:
# Define the getter function
def get_has_number(doc):
    # Return if any of the tokens in the doc return True for token.like_num
    return any(token.like_num for token in doc)

# Register the Doc property extension 'has_number' with the getter get_has_number
Doc.set_extension('has_number', getter=get_has_number)

# Process the text and check the custom has_number attribute 
doc = nlp("The museum closed for five years in 2012.")
print('has_number:', doc._.has_number)

has_number: True


Use Span.set_extension to register 'to_html' (method to_html).

In [8]:
# Define the method
def to_html(span, tag):
    # Wrap the span text in a HTML tag and return it
    return '<{tag}>{text}</{tag}>'.format(tag=tag, text=span.text)

# Register the Span property extension 'to_html' with the method to_html
Span.set_extension('to_html', method=to_html)

# Process the text and call the to_html method on the span with the tag name 'strong'
doc = nlp("Hello world, this is a sentence.")
span = doc[0:2]
print(span._.to_html('strong'))

<strong>Hello world</strong>


## Entities and extensions

In this exercise, you'll combine custom extension attributes with the model's predictions and create an attribute getter that returns a Wikipedia search URL if the span is a person, organization, or location.

In [9]:
# Define the custom component
def length_component(doc):
    # Get the doc's length
    doc_length = len(doc)
    print("This document is {} tokens long.".format(doc_length))
    # Return the doc
    return doc

# Load the small English model
nlp = spacy.load('en_core_web_sm')

# Add the component first in the pipeline and print the pipe names
nlp.add_pipe(length_component, first=True)
print(nlp.pipe_names)

['length_component', 'tagger', 'parser', 'ner']


**Example:** Mahatma Gandhi

In [10]:
# Load the small English model and Add the component first in the pipeline
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(length_component, first=True)

# Process a text
doc = nlp("Gandhi's birthday, 2 October, is commemorated in India as Gandhi Jayanti, a national holiday, and worldwide as the International Day of Nonviolence.")

This document is 28 tokens long.


In [11]:
def get_wikipedia_url(span):
    # Get a Wikipedia URL if the span has one of the labels
    if span.label_ in ('PERSON', 'ORG', 'GPE', 'LOCATION'):
        entity_text = span.text.replace(' ', '_')
        return "https://en.wikipedia.org/w/index.php?search=" + entity_text

# Set the Span extension wikipedia_url using get getter get_wikipedia_url
Span.set_extension('wikipedia_url', getter=get_wikipedia_url, force=True)

doc = nlp("Gandhi's birthday, 2 October, is commemorated in India as Gandhi Jayanti, a national holiday, and worldwide as the International Day of Nonviolence.")
for ent in doc.ents:
    # Print the text and Wikipedia URL of the entity
    print(ent.text, ent._.wikipedia_url)

This document is 28 tokens long.
Gandhi https://en.wikipedia.org/w/index.php?search=Gandhi
2 October None
India https://en.wikipedia.org/w/index.php?search=India
Gandhi Jayanti https://en.wikipedia.org/w/index.php?search=Gandhi_Jayanti
the International Day of Nonviolence None


## Components with extensions

Extension attributes are especially powerful if they're combined with custom pipeline components. In this exercise, you'll write a pipeline component that finds country names and a custom extension attribute that returns a country's capital, if available.

In [12]:
def countries_component(doc):
    # Create an entity Span with the label 'GPE' for all matches
    doc.ents = [Span(doc, start, end, label='GPE')
                for match_id, start, end in matcher(doc)]
    return doc

# Add the component to the pipeline
nlp.add_pipe(countries_component)
print(nlp.pipe_names)

['length_component', 'tagger', 'parser', 'ner', 'countries_component']


In [13]:
# Getter that looks up the span text in the dictionary of country capitals
get_capital = lambda span: capitals.get(span.text)

# Register the Span extension attribute 'capital' with the getter get_capital 
Span.set_extension('capital', getter=get_capital)

In [14]:
COUNTRIES = ['Afghanistan',
 'Åland Islands',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bonaire, Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory',
 'United States Minor Outlying Islands',
 'Virgin Islands (British)',
 'Virgin Islands (U.S.)',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cabo Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos (Keeling) Islands',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo (Democratic Republic of the)',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Falkland Islands (Malvinas)',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'French Polynesia',
 'French Southern Territories',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Grenada',
 'Guadeloupe',
 'Guam',
 'Guatemala',
 'Guernsey',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Heard Island and McDonald Islands',
 'Holy See',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 "Côte d'Ivoire",
 'Iran',
 'Iraq',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jersey',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kiribati',
 'Kuwait',
 'Kyrgyzstan',
 "Lao People's Democratic Republic",
 'Latvia',
 'Lebanon',
 'Lesotho',
 'Liberia',
 'Libya',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Macao',
 'Macedonia (the former Yugoslav Republic of)',
 'Madagascar',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Mali',
 'Malta',
 'Marshall Islands',
 'Martinique',
 'Mauritania',
 'Mauritius',
 'Mayotte',
 'Mexico',
 'Micronesia (Federated States of)',
 'Moldova (Republic of)',
 'Monaco',
 'Mongolia',
 'Montenegro',
 'Montserrat',
 'Morocco',
 'Mozambique',
 'Myanmar',
 'Namibia',
 'Nauru',
 'Nepal',
 'Netherlands',
 'New Caledonia',
 'New Zealand',
 'Nicaragua',
 'Niger',
 'Nigeria',
 'Niue',
 'Norfolk Island',
 "Korea (Democratic People's Republic of)",
 'Northern Mariana Islands',
 'Norway',
 'Oman',
 'Pakistan',
 'Palau',
 'Palestine, State of',
 'Panama',
 'Papua New Guinea',
 'Paraguay',
 'Peru',
 'Philippines',
 'Pitcairn',
 'Poland',
 'Portugal',
 'Puerto Rico',
 'Qatar',
 'Republic of Kosovo',
 'Réunion',
 'Romania',
 'Russian Federation',
 'Rwanda',
 'Saint Barthélemy',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Martin (French part)',
 'Saint Pierre and Miquelon',
 'Saint Vincent and the Grenadines',
 'Samoa',
 'San Marino',
 'Sao Tome and Principe',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'Seychelles',
 'Sierra Leone',
 'Singapore',
 'Sint Maarten (Dutch part)',
 'Slovakia',
 'Slovenia',
 'Solomon Islands',
 'Somalia',
 'South Africa',
 'South Georgia and the South Sandwich Islands',
 'Korea (Republic of)',
 'South Sudan',
 'Spain',
 'Sri Lanka',
 'Sudan',
 'Suriname',
 'Svalbard and Jan Mayen',
 'Swaziland',
 'Sweden',
 'Switzerland',
 'Syrian Arab Republic',
 'Taiwan',
 'Tajikistan',
 'Tanzania, United Republic of',
 'Thailand',
 'Timor-Leste',
 'Togo',
 'Tokelau',
 'Tonga',
 'Trinidad and Tobago',
 'Tunisia',
 'Turkey',
 'Turkmenistan',
 'Turks and Caicos Islands',
 'Tuvalu',
 'Uganda',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom of Great Britain and Northern Ireland',
 'United States of America',
 'Uruguay',
 'Uzbekistan',
 'Vanuatu',
 'Venezuela (Bolivarian Republic of)',
 'Vietnam',
 'Wallis and Futuna',
 'Western Sahara',
 'Yemen',
 'Zambia',
 'Zimbabwe']

In [15]:
capitals = {'Afghanistan': 'Kabul',
 'Albania': 'Tirana',
 'Algeria': 'Algiers',
 'American Samoa': 'Pago Pago',
 'Andorra': 'Andorra la Vella',
 'Angola': 'Luanda',
 'Anguilla': 'The Valley',
 'Antarctica': '',
 'Antigua and Barbuda': "Saint John's",
 'Argentina': 'Buenos Aires',
 'Armenia': 'Yerevan',
 'Aruba': 'Oranjestad',
 'Australia': 'Canberra',
 'Austria': 'Vienna',
 'Azerbaijan': 'Baku',
 'Bahamas': 'Nassau',
 'Bahrain': 'Manama',
 'Bangladesh': 'Dhaka',
 'Barbados': 'Bridgetown',
 'Belarus': 'Minsk',
 'Belgium': 'Brussels',
 'Belize': 'Belmopan',
 'Benin': 'Porto-Novo',
 'Bermuda': 'Hamilton',
 'Bhutan': 'Thimphu',
 'Bolivia (Plurinational State of)': 'Sucre',
 'Bonaire, Sint Eustatius and Saba': 'Kralendijk',
 'Bosnia and Herzegovina': 'Sarajevo',
 'Botswana': 'Gaborone',
 'Bouvet Island': '',
 'Brazil': 'Brasília',
 'British Indian Ocean Territory': 'Diego Garcia',
 'Brunei': 'Bandar Seri Begawan',
 'Bulgaria': 'Sofia',
 'Burkina Faso': 'Ouagadougou',
 'Burundi': 'Bujumbura',
 'Cabo Verde': 'Praia',
 'Cambodia': 'Phnom Penh',
 'Cameroon': 'Yaoundé',
 'Canada': 'Ottawa',
 'Cayman Islands': 'George Town',
 'Central African Republic': 'Bangui',
 'Chad': "N'Djamena",
 'Chile': 'Santiago',
 'China': 'Beijing',
 'Christmas Island': 'Flying Fish Cove',
 'Cocos (Keeling) Islands': 'West Island',
 'Colombia': 'Bogotá',
 'Comoros': 'Moroni',
 'Congo': 'Brazzaville',
 'Congo (Democratic Republic of the)': 'Kinshasa',
 'Cook Islands': 'Avarua',
 'Costa Rica': 'San José',
 'Croatia': 'Zagreb',
 'Cuba': 'Havana',
 'Curaçao': 'Willemstad',
 'Cyprus': 'Nicosia',
 'Czech Republic': 'Prague',
 "Côte d'Ivoire": 'Yamoussoukro',
 'Denmark': 'Copenhagen',
 'Djibouti': 'Djibouti',
 'Dominica': 'Roseau',
 'Dominican Republic': 'Santo Domingo',
 'Ecuador': 'Quito',
 'Egypt': 'Cairo',
 'El Salvador': 'San Salvador',
 'Equatorial Guinea': 'Malabo',
 'Eritrea': 'Asmara',
 'Estonia': 'Tallinn',
 'Ethiopia': 'Addis Ababa',
 'Falkland Islands (Malvinas)': 'Stanley',
 'Faroe Islands': 'Tórshavn',
 'Fiji': 'Suva',
 'Finland': 'Helsinki',
 'France': 'Paris',
 'French Guiana': 'Cayenne',
 'French Polynesia': 'Papeetē',
 'French Southern Territories': 'Port-aux-Français',
 'Gabon': 'Libreville',
 'Gambia': 'Banjul',
 'Georgia': 'Tbilisi',
 'Germany': 'Berlin',
 'Ghana': 'Accra',
 'Gibraltar': 'Gibraltar',
 'Greece': 'Athens',
 'Greenland': 'Nuuk',
 'Grenada': "St. George's",
 'Guadeloupe': 'Basse-Terre',
 'Guam': 'Hagåtña',
 'Guatemala': 'Guatemala City',
 'Guernsey': 'St. Peter Port',
 'Guinea': 'Conakry',
 'Guinea-Bissau': 'Bissau',
 'Guyana': 'Georgetown',
 'Haiti': 'Port-au-Prince',
 'Heard Island and McDonald Islands': '',
 'Holy See': 'Rome',
 'Honduras': 'Tegucigalpa',
 'Hong Kong': 'City of Victoria',
 'Hungary': 'Budapest',
 'Iceland': 'Reykjavík',
 'India': 'New Delhi',
 'Indonesia': 'Jakarta',
 'Iran': 'Tehran',
 'Iraq': 'Baghdad',
 'Ireland': 'Dublin',
 'Isle of Man': 'Douglas',
 'Israel': 'Jerusalem',
 'Italy': 'Rome',
 'Jamaica': 'Kingston',
 'Japan': 'Tokyo',
 'Jersey': 'Saint Helier',
 'Jordan': 'Amman',
 'Kazakhstan': 'Astana',
 'Kenya': 'Nairobi',
 'Kiribati': 'South Tarawa',
 "Korea (Democratic People's Republic of)": 'Pyongyang',
 'Korea (Republic of)': 'Seoul',
 'Kuwait': 'Kuwait City',
 'Kyrgyzstan': 'Bishkek',
 "Lao People's Democratic Republic": 'Vientiane',
 'Latvia': 'Riga',
 'Lebanon': 'Beirut',
 'Lesotho': 'Maseru',
 'Liberia': 'Monrovia',
 'Libya': 'Tripoli',
 'Liechtenstein': 'Vaduz',
 'Lithuania': 'Vilnius',
 'Luxembourg': 'Luxembourg',
 'Macao': '',
 'Macedonia (the former Yugoslav Republic of)': 'Skopje',
 'Madagascar': 'Antananarivo',
 'Malawi': 'Lilongwe',
 'Malaysia': 'Kuala Lumpur',
 'Maldives': 'Malé',
 'Mali': 'Bamako',
 'Malta': 'Valletta',
 'Marshall Islands': 'Majuro',
 'Martinique': 'Fort-de-France',
 'Mauritania': 'Nouakchott',
 'Mauritius': 'Port Louis',
 'Mayotte': 'Mamoudzou',
 'Mexico': 'Mexico City',
 'Micronesia (Federated States of)': 'Palikir',
 'Moldova (Republic of)': 'Chișinău',
 'Monaco': 'Monaco',
 'Mongolia': 'Ulan Bator',
 'Montenegro': 'Podgorica',
 'Montserrat': 'Plymouth',
 'Morocco': 'Rabat',
 'Mozambique': 'Maputo',
 'Myanmar': 'Naypyidaw',
 'Namibia': 'Windhoek',
 'Nauru': 'Yaren',
 'Nepal': 'Kathmandu',
 'Netherlands': 'Amsterdam',
 'New Caledonia': 'Nouméa',
 'New Zealand': 'Wellington',
 'Nicaragua': 'Managua',
 'Niger': 'Niamey',
 'Nigeria': 'Abuja',
 'Niue': 'Alofi',
 'Norfolk Island': 'Kingston',
 'Northern Mariana Islands': 'Saipan',
 'Norway': 'Oslo',
 'Oman': 'Muscat',
 'Pakistan': 'Islamabad',
 'Palau': 'Ngerulmud',
 'Palestine, State of': 'Ramallah',
 'Panama': 'Panama City',
 'Papua New Guinea': 'Port Moresby',
 'Paraguay': 'Asunción',
 'Peru': 'Lima',
 'Philippines': 'Manila',
 'Pitcairn': 'Adamstown',
 'Poland': 'Warsaw',
 'Portugal': 'Lisbon',
 'Puerto Rico': 'San Juan',
 'Qatar': 'Doha',
 'Republic of Kosovo': 'Pristina',
 'Romania': 'Bucharest',
 'Russian Federation': 'Moscow',
 'Rwanda': 'Kigali',
 'Réunion': 'Saint-Denis',
 'Saint Barthélemy': 'Gustavia',
 'Saint Helena, Ascension and Tristan da Cunha': 'Jamestown',
 'Saint Kitts and Nevis': 'Basseterre',
 'Saint Lucia': 'Castries',
 'Saint Martin (French part)': 'Marigot',
 'Saint Pierre and Miquelon': 'Saint-Pierre',
 'Saint Vincent and the Grenadines': 'Kingstown',
 'Samoa': 'Apia',
 'San Marino': 'City of San Marino',
 'Sao Tome and Principe': 'São Tomé',
 'Saudi Arabia': 'Riyadh',
 'Senegal': 'Dakar',
 'Serbia': 'Belgrade',
 'Seychelles': 'Victoria',
 'Sierra Leone': 'Freetown',
 'Singapore': 'Singapore',
 'Sint Maarten (Dutch part)': 'Philipsburg',
 'Slovakia': 'Bratislava',
 'Slovenia': 'Ljubljana',
 'Solomon Islands': 'Honiara',
 'Somalia': 'Mogadishu',
 'South Africa': 'Pretoria',
 'South Georgia and the South Sandwich Islands': 'King Edward Point',
 'South Sudan': 'Juba',
 'Spain': 'Madrid',
 'Sri Lanka': 'Colombo',
 'Sudan': 'Khartoum',
 'Suriname': 'Paramaribo',
 'Svalbard and Jan Mayen': 'Longyearbyen',
 'Swaziland': 'Lobamba',
 'Sweden': 'Stockholm',
 'Switzerland': 'Bern',
 'Syrian Arab Republic': 'Damascus',
 'Taiwan': 'Taipei',
 'Tajikistan': 'Dushanbe',
 'Tanzania, United Republic of': 'Dodoma',
 'Thailand': 'Bangkok',
 'Timor-Leste': 'Dili',
 'Togo': 'Lomé',
 'Tokelau': 'Fakaofo',
 'Tonga': "Nuku'alofa",
 'Trinidad and Tobago': 'Port of Spain',
 'Tunisia': 'Tunis',
 'Turkey': 'Ankara',
 'Turkmenistan': 'Ashgabat',
 'Turks and Caicos Islands': 'Cockburn Town',
 'Tuvalu': 'Funafuti',
 'Uganda': 'Kampala',
 'Ukraine': 'Kiev',
 'United Arab Emirates': 'Abu Dhabi',
 'United Kingdom of Great Britain and Northern Ireland': 'London',
 'United States Minor Outlying Islands': '',
 'United States of America': 'Washington, D.C.',
 'Uruguay': 'Montevideo',
 'Uzbekistan': 'Tashkent',
 'Vanuatu': 'Port Vila',
 'Venezuela (Bolivarian Republic of)': 'Caracas',
 'Vietnam': 'Hanoi',
 'Virgin Islands (British)': 'Road Town',
 'Virgin Islands (U.S.)': 'Charlotte Amalie',
 'Wallis and Futuna': 'Mata-Utu',
 'Western Sahara': 'El Aaiún',
 'Yemen': "Sana'a",
 'Zambia': 'Lusaka',
 'Zimbabwe': 'Harare',
 'Åland Islands': 'Mariehamn'}

In [16]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [17]:
# Import the PhraseMatcher and initialize it
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

# Create pattern Doc objects and add them to the matcher
# This is the faster version of: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add('COUNTRY', None, *patterns)

# Call the matcher on the test document and print the result
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

This document is 1 tokens long.
This document is 2 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 2 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 3 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 1 tokens long.
This document is 6 tokens long.
This document is 6 tokens long.
This document is 3 tokens long.
This document is 1 tokens long.
This document is 2 tokens long.
This doc

In [18]:
# Process the text and print the entity text, label and capital attributes
doc = nlp("In the late 1990s and early 2000s, the economies of China and India have been growing rapidly, both with an average annual growth rate of more than 8%. Other recent very-high-growth nations in Asia include Israel, Malaysia, Indonesia, Bangladesh, Thailand, Vietnam, and the Philippines, and mineral-rich nations such as Kazakhstan, Turkmenistan, Iran, Brunei, the United Arab Emirates, Qatar, Kuwait, Saudi Arabia, Bahrain and Oman.")
print([(ent.text, ent.label_, ent._.capital) for ent in doc.ents])

This document is 90 tokens long.
[('China', 'GPE', 'Beijing'), ('India', 'GPE', 'New Delhi'), ('Israel', 'GPE', 'Jerusalem'), ('Malaysia', 'GPE', 'Kuala Lumpur'), ('Indonesia', 'GPE', 'Jakarta'), ('Bangladesh', 'GPE', 'Dhaka'), ('Thailand', 'GPE', 'Bangkok'), ('Vietnam', 'GPE', 'Hanoi'), ('Philippines', 'GPE', 'Manila'), ('Kazakhstan', 'GPE', 'Astana'), ('Turkmenistan', 'GPE', 'Ashgabat'), ('Iran', 'GPE', 'Tehran'), ('Brunei', 'GPE', 'Bandar Seri Begawan'), ('United Arab Emirates', 'GPE', 'Abu Dhabi'), ('Qatar', 'GPE', 'Doha'), ('Kuwait', 'GPE', 'Kuwait City'), ('Saudi Arabia', 'GPE', 'Riyadh'), ('Bahrain', 'GPE', 'Manama'), ('Oman', 'GPE', 'Muscat')]
